In [12]:
import numpy as np
import tensorflow as tf
import pandas as pd
import yfinance as yf
import sys
from tqdm import tqdm

In [2]:
class Brain:
    def __init__(self, input_dim=(15,6)):
        self.model = tf.keras.Sequential ([
            tf.keras.layers.InputLayer(input_shape=input_dim),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(100, activation='relu'),
            tf.keras.layers.Dense(100, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

    def predict(self, tensor):
        if len(tensor.shape) == 2:
            tensor = tf.expand_dims(tensor, axis=0)
        predictions = self.model.predict(tensor, verbose=0)
        return (predictions > 0.5).astype(int)

    def get_weights(self):
        return self.model.weights

    def set_weights(self, tensor):
        self.model.set_weights(tensor)

test_input = np.random.random((1, 15, 6))

brain = Brain()
output = brain.predict(test_input)
print("Output shape:", output.shape)
print("Output:", output)

Output shape: (1, 1)
Output: [[1]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [3]:
class Data:
    def __init__(self, start_index=0):
        self.data = yf.download('AAPL', period='5y', interval='1d')
        self.current_index = start_index
        self.interval_data = tf.convert_to_tensor(self.data.iloc[self.current_index], dtype=tf.float32)
        self.current_price = self.data.iloc[self.current_index][0]
        self.length = self.data.shape[0]

    def get_interval_data(self):
        return self.interval_data

    def get_current_price(self):
        return self.data.iloc[self.current_index][0]

    def next(self):
        if self.current_index < self.data.shape[0]:
            self.current_index = self.current_index + 1
            self.interval_data = tf.convert_to_tensor(self.data.iloc[self.current_index], dtype=tf.float32)
        else:
            self.current_index = 0
            self.interval_data = tf.convert_to_tensor(self.data.iloc[self.current_index], dtype=tf.float32)


data_obj = Data()
data_obj.data[:5]
print(data_obj.get_interval_data())
data_obj.next()
print(data_obj.get_interval_data())
print(data_obj.get_current_price())

[*********************100%%**********************]  1 of 1 completed

tf.Tensor(
[5.0099998e+01 5.0419998e+01 4.9165001e+01 5.0180000e+01 4.8308235e+01
 1.3963440e+08], shape=(6,), dtype=float32)
tf.Tensor(
[4.9355000e+01 4.9712502e+01 4.8192501e+01 4.9294998e+01 4.7638996e+01
 1.6483480e+08], shape=(6,), dtype=float32)
49.35499954223633



/var/folders/fn/vxn26nqn3qj3s7f8868n0x5c0000gn/T/ipykernel_57868/3252891350.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.current_price = self.data.iloc[self.current_index][0]
/var/folders/fn/vxn26nqn3qj3s7f8868n0x5c0000gn/T/ipykernel_57868/3252891350.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.data.iloc[self.current_index][0]


In [4]:
class SlidingWindow:
    def __init__(self, input_dim):
        self.sliding_window = tf.Variable(tf.zeros(input_dim, dtype=tf.float32))
        self.interval_data_size = input_dim[1]
        self.days = input_dim[0]

    def __repr__(self):
        return str(self.sliding_window.numpy())

    def insert(self, tensor):
        if tf.size(tf.constant(tensor)).numpy() != self.interval_data_size:
            print(f"Error: trying to insert tensor of size {tf.size(tensor).numpy()}. Expected size of {self.interval_data_size}.")
        else:
            tensor = tf.cast(tensor, tf.float32)
            self.sliding_window.assign(tf.concat([tf.expand_dims(tensor, 0), self.sliding_window[:-1]], axis=0))

tensor_to_insert = tf.constant([1, 2, 3, 4, 5, 6], dtype=tf.float32)
another_tensor_to_insert = tf.constant([6, 7, 8, 9, 10, 11], dtype=tf.float32)
input_dim = (15, 6)
interval_data_size = tf.size(tensor_to_insert).numpy()

sl_win = SlidingWindow(input_dim)
print(sl_win)
sl_win.insert(tensor_to_insert)
print(sl_win)
sl_win.insert(another_tensor_to_insert)
print(sl_win)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[1. 2. 3. 4. 5. 6.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[ 6.  7.  8.  9. 10. 11.]
 [ 1.  2.  3.  4.  5.  6.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.

In [5]:
class Agent:
    def __init__(self):
        self.active_trades = []
        self.fitness = 0
        self.num_trades = 0
        self.brain = Brain()
        self.sl_win = SlidingWindow((15,6))

    def __lt__(self, other):
        return True if self.fitness < other.fitness else False

    def update(self, tensor, current_price):
        self.sl_win.insert(tensor)
        self.close_trades(current_price)
        if self.brain.predict(sl_win.sliding_window) > .5:
            self.buy(current_price)

    def buy(self, current_price):
        self.active_trades.append([current_price, current_price * 1.05, current_price * .95])
        self.num_trades = self.num_trades + 1

    def close_trades(self, current_price):
        trades_to_keep = []
        for trade in self.active_trades:
            if trade[1] <= current_price:
                self.fitness = self.fitness + trade[1] - trade[0]
            elif trade[2] >= current_price:
                self.fitness = self.fitness + trade[0] - trade[2]
            else:
                trades_to_keep.append(trade)
        self.active_trades = trades_to_keep

    def force_close_trades(self, current_price):
        trades_to_keep = []
        for trade in self.active_trades:
            self.fitness = self.fitness + current_price - trade[0]
        self.active_trades = trades_to_keep

In [ ]:
class Gen:
    def __init__(self, generations, population_size, cycles=100, mutation_rate=0.03):
        self.generations = generations
        self.population_size = population_size
        self.cycles = cycles
        self.mutation_rate = mutation_rate

    def mutate(self, child):
        pass

    def crossover(self, parent1, parent2):
        pass

    def gen(self):
        data = Data()
        data.data, test_set = data.data[:-200], data.data[-200:]
        population = []
        for _ in range(self.population_size):
            population.append(Agent())
        for generation in range(self.generations):
            for i in range(self.cycles):
                if (i + 1) % 10 == 0:
                    print(f"Cycle {i+1}/{self.cycles}")
                input_tensor = data.get_interval_data()
                for agent in population:
                    agent.update(input_tensor, data.get_current_price())
                data.next()
                sys.stdout.flush()
                
            ranked_population = sorted(population)
            ranked_population.reverse()

            print(f"=== Generation {generation}===\nBest: {ranked_population[0].fitness}. Worst{ranked_population[-1].fitness}")

            # pick parents
            # make children
            # mutate children
            # assign new generation to population

genetic_alg = Gen(1, 10, 100, .03)
genetic_alg.gen()

[*********************100%%**********************]  1 of 1 completed
/var/folders/fn/vxn26nqn3qj3s7f8868n0x5c0000gn/T/ipykernel_57868/3252891350.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.current_price = self.data.iloc[self.current_index][0]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/var/folders/fn/vxn26nqn3qj3s7f8868n0x5c0000gn/T/ipykernel_57868/3252891350.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.data.iloc[self.cu

Cycle 10/100
Cycle 20/100
Cycle 30/100
